In [ ]:
from renderer import Chat
chat = Chat.fromKey("1096206")
doc = chat.build()
with open("test.md", "w") as f:
    f.writelines(doc.render())

In [ ]:
with open("foobar.json", "w") as f:
    from renderer import DB
    import json
    json.dump(DB.getDocument("chat-logs", "1096206"), f)